In [1]:
import time

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="")

In [2]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

openai_api_key = ''  # Provide the API key as a string


embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)



In [3]:
import os

# 此处要把API key set 进环境里 Set your Pinecone API key
os.environ['PINECONE_API_KEY'] = ''


In [4]:
api_key = os.environ.get("PINECONE_API_KEY")

# available at platform.openai.com/api-keys
os.environ['OPENAI_API_KEY'] =  ""

In [5]:
from langchain_text_splitters import MarkdownHeaderTextSplitter


markdown_document = """## Introduction

Welcome to the dynamic and integrative world of Cooperative Education at Northeastern University, a premier program designed to weave real-world experience with academic study across a variety of global industries. This program is not just about gaining work experience, but about creating a tapestry of learning that enhances your academic journey and prepares you for a successful career. Dive into the depths of this innovative program, where practical experience meets academic rigor to create a unique and enriching educational adventure.

## Program Overview

Northeastern's Cooperative Education program, or co-op, is a cornerstone of the university's experiential learning model. Students alternate between academic study and full-time employment, gaining experience in fields directly related to their academic and career interests. This blend of classroom and practical learning provides a holistic education that enhances both personal and professional growth.

### Key Features

- **Full-Time Employment**: Engage in full-time work in positions that complement your academic pursuits.
- **Global Opportunities**: Explore work opportunities across the globe, applying your knowledge in diverse cultural and professional settings.
- **Integrated Learning**: Apply classroom theories to practical challenges in the workplace, enhancing your academic knowledge and professional skills simultaneously.

### General Requirements

- **Enrollment**: Must be enrolled as a full-time student.
- **Academic Standing**: Complete at least two full-time semesters prior to your first co-op.
- **Pre-Co-op Preparation**: Fulfill all college-specific pre-co-op requirements.

### Special Provisions for Diverse Student Groups

- **Transfer Students**: Complete similar pre-co-op prerequisites and at least one semester at Northeastern.
- **International Students**: Fulfill one academic year and secure proper work authorization.

## Academic Requirements

Experience work in a full-time role, defined by either one full-time job or multiple part-time positions, ensuring a minimum of 32-40 hours per week. The co-op is assessed on a Satisfactory/Unsatisfactory basis, influencing your academic standing and professional trajectory.

## Getting Started

Embark on your co-op journey by following these whimsical yet practical steps:

1. **Validation of Eligibility**: Meet with a co-op coordinator to verify your readiness and eligibility.
2. **Position Selection**: Utilize Northeastern's extensive job portal to find and secure a position that aligns with your career goals.
3. **Official Registration**: Register your co-op position through the university’s designated system, ensuring all details are correctly documented.

## Troubleshooting Common Co-op Conundrums

Even in a well-structured program like co-op, challenges may arise. Here are some quick fixes:

- **Problem: Difficulty in Securing a Position**:
  - **Magic Fix**: Enhance your resume and interview skills with help from career services.
  
- **Problem: Co-op Registration Errors**:
  - **Magic Fix**: Double-check all details for accuracy and consult your co-op advisor for immediate rectification.
  
- **Problem: Work Authorization Issues (International Students)**:
  - **Magic Fix**: Contact the Office of Global Services well in advance to ensure all paperwork is flawless.

## Conclusion

Step into the world of Cooperative Education at Northeastern University where academic theories and professional realities intermingle to prepare you for a future as vibrant and dynamic as the co-op program itself. Forge your path, gather invaluable experiences, and build a robust professional network through a program designed to elevate your educational journey."""


headers_to_split_on = [
    ("##", "Header 2")
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

print(md_header_splits)

[Document(metadata={'Header 2': 'Introduction'}, page_content='## Introduction  \nWelcome to the dynamic and integrative world of Cooperative Education at Northeastern University, a premier program designed to weave real-world experience with academic study across a variety of global industries. This program is not just about gaining work experience, but about creating a tapestry of learning that enhances your academic journey and prepares you for a successful career. Dive into the depths of this innovative program, where practical experience meets academic rigor to create a unique and enriching educational adventure.'), Document(metadata={'Header 2': 'Program Overview'}, page_content="## Program Overview  \nNortheastern's Cooperative Education program, or co-op, is a cornerstone of the university's experiential learning model. Students alternate between academic study and full-time employment, gaining experience in fields directly related to their academic and career interests. This b

In [6]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-3-small'
embeddings = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=''
)

In [7]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='')

In [8]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [9]:
index_name = "chatbot-v2"

In [10]:
import time

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=spec
    )
    # wait for index to be ready
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

In [11]:
from langchain_pinecone import PineconeVectorStore

namespace = "wondervector5000"

docsearch = PineconeVectorStore.from_documents(
    documents=md_header_splits,
    index_name=index_name,
    embedding=embeddings,
    namespace=namespace
)

time.sleep(1)

In [12]:
index = pc.Index(index_name)

for ids in index.list(namespace=namespace):
    query = index.query(
        id=ids[0],
        namespace=namespace,
        top_k=1,
        include_values=True,
        include_metadata=True
    )
    print(query)

{'matches': [{'id': '057dc98e-d1c7-4102-97b0-e3f136c1b85a',
              'metadata': {'Header 2': 'Use cases',
                           'text': '## Use cases  \n'
                                   'While the WonderVector5000 is '
                                   'fundamentally a device of fiction and fun, '
                                   "let's imagine some scenarios where it "
                                   'could hypothetically be applied:  \n'
                                   '- Time Travel Adventures: Use the '
                                   'Chrono-Distortion Field to visit key '
                                   'moments in history or glimpse into the '
                                   'future. While actual temporal manipulation '
                                   'is impossible, the mere idea sparks '
                                   'endless storytelling possibilities.  \n'
                                   '- Interdimensional Gaming: Engage with the 

In [13]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    openai_api_key=os.environ.get('OPENAI_API_KEY'),
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [14]:
query1 = "What are the Academic Requirements of northeastern university Cooperative Education??"

query2 = "How do I registration for Co-op?"

In [15]:
query1_with_knowledge = qa.invoke(query1)
query1_without_knowledge = llm.invoke(query1)

print(query1_with_knowledge)
print()
print(query1_without_knowledge)

{'query': 'What are the Academic Requirements of northeastern university Cooperative Education??', 'result': "The academic requirements for Northeastern University's Cooperative Education program include being enrolled as a full-time student, completing at least two full-time semesters prior to your first co-op, and fulfilling all college-specific pre-co-op requirements."}

content="The academic requirements for Northeastern University's Cooperative Education program vary depending on the specific program and major. However, in general, students must meet the following requirements:\n\n1. Students must be enrolled in a full-time undergraduate or graduate program at Northeastern University.\n2. Students must have completed a minimum number of credit hours (usually around 30-60 credits) before they are eligible to participate in the Cooperative Education program.\n3. Students must have a minimum GPA (usually around 2.5-3.0) to be eligible for the program.\n4. Students must attend mandato

In [16]:
query2_with_knowledge = qa.invoke(query2)
query2_without_knowledge = llm.invoke(query2)

print(query2_with_knowledge)
print()
print(query2_without_knowledge)

{'query': 'How do I registration for Co-op?', 'result': "To register for Co-op, you need to follow these steps:\n1. Meet with a co-op coordinator to verify your readiness and eligibility.\n2. Utilize Northeastern's job portal to find and secure a position that aligns with your career goals.\n3. Register your co-op position through the university’s designated system, ensuring all details are correctly documented."}

content="To register for a co-op program, you will typically need to follow these steps:\n\n1. Contact your school's career services or co-op office to inquire about available co-op opportunities and the registration process.\n\n2. Attend any information sessions or workshops provided by the co-op office to learn more about the program and its requirements.\n\n3. Complete any necessary application forms or online registration processes. This may include submitting your resume, cover letter, and any other required documents.\n\n4. Participate in any interviews or assessments 